# Interictal Analysis - Neural Fragility

In the fragility perturbation model, we traditionally perturb the columns of the linear system matrix. However, assuming the system is not symmetric, then a row-perturbation provides an alternative view on how epileptic a region might be hypothesized.

In [2]:
import os
import re
import json
import random
import collections
from pprint import pprint
from pathlib import Path
import sys

import numpy as np
import pandas as pd
import scipy

from eztrack.utils import warn
from mne_bids import read_raw_bids, BIDSPath, get_entity_vals
from natsort import natsorted, index_natsorted, order_by_index
import scipy
import math

sys.path.append('../')

from analysis.fragility.posthoc import read_perturbation_result

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# Load Data

In [4]:
WORKSTATION = "home"

if WORKSTATION == "home":
    # bids root to write BIDS data to
    # the root of the BIDS dataset
    root = Path("/Users/adam2392/Dropbox/epilepsy_bids/")
    output_dir = root / 'derivatives' / 'interictal'

    figures_dir = output_dir / 'figures'

    # path to excel layout file - would be changed to the datasheet locally
    excel_fpath = Path(
        "/Users/adam2392/Dropbox/epilepsy_bids/sourcedata/organized_clinical_datasheet_raw.xlsx"
    )
elif WORKSTATION == "lab":
    root = Path("/home/adam2392/hdd/epilepsy_bids/")
    excel_fpath = Path(
        "/home/adam2392/hdd/epilepsy_bids/sourcedata/organized_clinical_datasheet_raw.xlsx"
    )

    # output directory
    output_dir = Path("/home/adam2392/hdd/epilepsy_bids") / 'derivatives' / 'interictal'

    # figures directory
    figures_dir = output_dir / 'figures'
    

In [6]:
SUBJECTS = [
    'jh103',
    'jh105'
]

session = "presurgery"  # only one session
task = "interictal"
datatype = "ieeg"
acquisition = "ecog"  # or SEEG
extension = ".vhdr"

if acquisition == 'ecog':
    ignore_acquisitions = ['seeg']
elif acquisition == 'seeg':
    ignore_acquisitions = ['ecog']

reference = 'monopolar'
sfreq = None  # either resample or don't

# get the runs for this subject
all_subjects = get_entity_vals(root, "subject")

for subject in all_subjects:
    if subject not in SUBJECTS:
        continue
    ignore_subs = [sub for sub in all_subjects if sub != subject]
    all_tasks = get_entity_vals(root, "task", ignore_subjects=ignore_subs)
    ignore_tasks = [tsk for tsk in all_tasks if tsk != task]

    print(f"Analyzing {task} task for {subject}.")
    ignore_tasks = [tsk for tsk in all_tasks if tsk != task]
    runs = get_entity_vals(
        root, 'run', ignore_subjects=ignore_subs,
        ignore_tasks=ignore_tasks,
        ignore_acquisitions=ignore_acquisitions
    )
    print(f'Found {runs} runs for {task} task.')

    deriv_path = (output_dir
                  # /  'nodepth'
                  / '1000Hz'
                  / "fragility"
                  / reference
                  / f"sub-{subject}")
    
    for idx, run in enumerate(runs):
        # create path for the dataset
        bids_path = BIDSPath(
            subject=subject,
            session=session,
            task=task,
            run=run,
            datatype=datatype,
            acquisition=acquisition,
            suffix=datatype,
            root=root,
#             extension=extension,
        )
        source_basename = bids_path.basename
        description = 'perturbmatrix'
        col_result = read_perturbation_result(deriv_path, source_basename, description)
        
        description = 'rowperturbmatrix'
        row_result = read_perturbation_result(deriv_path, source_basename, description)
        
        break
    break

Analyzing interictal task for jh103.
Found ['01', '02'] runs for interictal task.
TRYING TO READ RESULT FILENAME  sub-jh103_ses-presurgery_task-interictal_acq-ecog_run-01_desc-rowperturbmatrix_ieeg.npy
Loading /Users/adam2392/Dropbox/epilepsy_bids/derivatives/interictal/1000Hz/fragility/monopolar/sub-jh103/sub-jh103_ses-presurgery_task-interictal_acq-ecog_run-01_desc-rowperturbmatrix_ieeg.npy, which exists: True
TRYING TO READ RESULT FILENAME  sub-jh103_ses-presurgery_task-interictal_acq-ecog_run-01_desc-rowperturbmatrix_ieeg.npy
Loading /Users/adam2392/Dropbox/epilepsy_bids/derivatives/interictal/1000Hz/fragility/monopolar/sub-jh103/sub-jh103_ses-presurgery_task-interictal_acq-ecog_run-01_desc-rowperturbmatrix_ieeg.npy, which exists: True
